In [1]:
# dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 3 - Tabelas tópico 3.5

### Geração de Massa

In [2]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.faixaRiscoInternoAgrupada':["R10"],
                             'payloadHomol.intermediarias.segmentacaoPolitica':["P1 - Funcionarios"],
                             'payloadHomol.intermediarias.faixaRendaBruta':["d. >= 15k"],
                             'solicitante.flagFuncionario':[0],
                             'payloadHomol.intermediarias.cpf6e7Digito':[99],
                             'solicitante.flagCCRot':[1],
                             'solicitante.mediaSaldoTotal':[40000]
                            })

In [4]:
#Criação das massas com casos forçados
massa_TabelaJaCliente1 = sherlock.Gera_Massa(base_entrada)
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.faixaRiscoInternoAgrupada',["R1","R2","R3",
                                                                                            "R4","R5","R6",
                                                                                            "R7","R8","R9",
                                                                                            "R10","R11","R12"])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.segmentacaoPolitica',["P1 - Funcionarios",
                                                                                      "P2 - FOPAG",
                                                                                      "P3 - Investidor Alta Renda",
                                                                                      "P7.1 – Jovem Cliente Principalidade",
                                                                                      "P7.2 – Jovem Cliente MAT",
                                                                                      "P7.3 – Jovem Cliente Inativos",
                                                                                      "P4.1 - Alta Renda Principalidade",
                                                                                      "P4.2 - Alta Renda MAT",
                                                                                      "P4.3 - Alta Renda Inativo",
                                                                                      "P5.4 - Varejo+ Principalidade Renda < 4k",
                                                                                      "P5.1 - Varejo+ Principalidade Renda >= 4k",
                                                                                      "P5.2 - Varejo+ MAT",
                                                                                      "P5.3 - Varejo+ Inativos",
                                                                                      "P6.1 - Varejo MAT",
                                                                                      "P6.2 - Varejo Inativos"])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.faixaRendaBruta',['a. Sem renda',
                                                                                  'a. < 4k',
                                                                                  'b. 4k - 10k',
                                                                                  'c. 10k - 15k',
                                                                                  'd. >= 15k',
                                                                                  'z. Verificar'])
massa_TabelaJaCliente1.gera_cenario('solicitante.flagFuncionario',[0,1])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.cpf6e7Digito',[0,43,44,45,46,88,89,90,91,99])
# massa_TabelaJaCliente1 = massa_TabelaJaCliente1.get_massa_final()

# # Teste de aplicação da coluna de rotativo e mean total balance para clientes investidor alta renda
# massa_TabelaJaCliente2 = sherlock.Gera_Massa(base_entrada)
# massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.faixaRiscoInternoAgrupada',["R1","R2","R3",
#                                                                                             "R4","R5","R6",
#                                                                                             "R7","R8","R9",
#                                                                                             "R10"])
# massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.segmentacaoPolitica',["P1 - Funcionarios",
#                                                                                        "P3 - Investidor Alta Renda"
#                                                                                        ])
# massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.faixaRendaBruta',['a. Sem renda',
#                                                                                    'a. < 4k',
#                                                                                    'd. >= 15k'])
# massa_TabelaJaCliente2.gera_cenario('solicitante.flagFuncionario',[0,1])
# massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.cpf6e7Digito',[0,99])
# massa_TabelaJaCliente2.gera_cenario('solicitante.flagCCRot',[0,1])
# massa_TabelaJaCliente2.gera_cenario('solicitante.mediaSaldoTotal',[39999,40000,40001])
# massa_TabelaJaCliente2 = massa_TabelaJaCliente2.get_massa_final()

#Empilha todos casos com 1 caso padrão.
# massa_TabelaJaCliente = sherlock.Gera_Massa(base_entrada)
# massa_TabelaJaCliente.empilha_massas([massa_TabelaJaCliente1,
#                                       massa_TabelaJaCliente2
# ])

# massa_TabelaJaCliente.get_massa_final()

massa_TabelaJaCliente1.set_bom_arqv()
massa_TabelaJaCliente1.converte_massa()

--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


### Execução Modular e Geração de CSV

In [5]:
listaResultado = rp.execucaoModular(massa_TabelaJaCliente1.get_massa_json(),rp.subGruposAntigo)

##### Valida se todas condições estão sendo abordadas

In [6]:
# buscando um perfil de cliente para análise
for payload in listaResultado:
    if (payload['payloadHomol']['intermediarias']['faixaRiscoInternoAgrupada']  == "R7"
        and
        payload['payloadHomol']['intermediarias']['segmentacaoPolitica'] == "P7.3 – Jovem Cliente Inativos"
        and 
        payload['payloadHomol']['intermediarias']['faixaRendaBruta'] == "a. Sem renda"
        and 
        payload['payloadHomol']['intermediarias']['cpf6e7Digito'] == "91"
        and
        payload['solicitante']['flagFuncionario'] == 0
        and 
        payload['solicitante']['flagCCRot'] == 1
        ):
        print(json.dumps(payload,indent = 4))
        break

In [7]:
# mudando o max de display de linhas dos dataframes desse notebook
pd.set_option('display.max_rows', 300) 
import pandas as pd

# fazendo freq para analisar se todos os cenários da tabela são abordados na massa
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['cenario'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha.sort_values(by='valor', ascending=True)

,valor,Freq
1,1,10
10,2,10
19,3,10
28,4,10
37,5,10
47,6,10
62,7,10
77,8,10
92,9,10
107,10,10


In [8]:
# Renomeação das colunas para o cabeçalho do RMA
massa_TabelaJaCliente1.rename_cabecalho()
massa_TabelaJaCliente1.rename_variavel("solicitante.flagCCRot","payloadParam.solicitante.flagCCRot")
massa_TabelaJaCliente1.rename_variavel("solicitante.mediaSaldoTotal","payloadParam.solicitante.mediaSaldoTotal")

In [9]:
massa_TabelaJaCliente1.get_massa_dm()

,Data-ID,Result-State,Result-Detail,payloadParam.solicitante.flagCCRot,payloadParam.solicitante.mediaSaldoTotal,rangeScoreInterno,payloadParamBTS.segmentacaoPolitica,faixaRendaBruta,payloadParam.solicitante.flagFuncionario,cpf6e7Digito
0,1,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,0
1,2,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,43
2,3,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,44
3,4,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,45
4,5,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,46
...,...,...,...,...,...,...,...,...,...,...
21595,21596,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,88
21596,21597,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,89
21597,21598,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,90
21598,21599,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,91


In [10]:
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacao","segmentacaoSubgrupo")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.alavancagem","alavancagem")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteTeto","limiteTeto")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.limitePiso","limitePiso")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteFixo","limiteFixo")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoBAUAux","aprovadoBAUAux")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoSLAux","aprovadoSmallLimitsAux")

massa_TabelaJaCliente1.get_massa_dm()

,Data-ID,Result-State,Result-Detail,payloadParam.solicitante.flagCCRot,payloadParam.solicitante.mediaSaldoTotal,rangeScoreInterno,payloadParamBTS.segmentacaoPolitica,faixaRendaBruta,payloadParam.solicitante.flagFuncionario,cpf6e7Digito,...,result:limiteTeto,expected:limiteTeto,result:limitePiso,expected:limitePiso,result:limiteFixo,expected:limiteFixo,result:aprovadoBAUAux,expected:aprovadoBAUAux,result:aprovadoSmallLimitsAux,expected:aprovadoSmallLimitsAux
0,1,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,0,...,,0.0,,0.0,,0,,0,,0
1,2,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,43,...,,0.0,,0.0,,0,,0,,0
2,3,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,44,...,,0.0,,0.0,,0,,0,,0
3,4,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,45,...,,0.0,,0.0,,0,,0,,0
4,5,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,46,...,,0.0,,0.0,,0,,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21595,21596,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,88,...,,0.0,,0.0,,0,,0,,0
21596,21597,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,89,...,,0.0,,0.0,,0,,0,,0
21597,21598,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,90,...,,0.0,,0.0,,0,,0,,0
21598,21599,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,91,...,,0.0,,0.0,,0,,0,,0


### Conversão da massa para CSV

In [11]:
massa_TabelaJaCliente1.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaDefinicaoTabelasSubGruposAntigosV2.csv',index=False)

In [12]:
dataframeGrande = massa_TabelaJaCliente1.get_massa_dm()
dataframeGrande

,Data-ID,Result-State,Result-Detail,payloadParam.solicitante.flagCCRot,payloadParam.solicitante.mediaSaldoTotal,rangeScoreInterno,payloadParamBTS.segmentacaoPolitica,faixaRendaBruta,payloadParam.solicitante.flagFuncionario,cpf6e7Digito,...,result:limiteTeto,expected:limiteTeto,result:limitePiso,expected:limitePiso,result:limiteFixo,expected:limiteFixo,result:aprovadoBAUAux,expected:aprovadoBAUAux,result:aprovadoSmallLimitsAux,expected:aprovadoSmallLimitsAux
0,1,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,0,...,,0.0,,0.0,,0,,0,,0
1,2,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,43,...,,0.0,,0.0,,0,,0,,0
2,3,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,44,...,,0.0,,0.0,,0,,0,,0
3,4,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,45,...,,0.0,,0.0,,0,,0,,0
4,5,,,1,40000,R1,P1 - Funcionarios,a. Sem renda,0,46,...,,0.0,,0.0,,0,,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21595,21596,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,88,...,,0.0,,0.0,,0,,0,,0
21596,21597,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,89,...,,0.0,,0.0,,0,,0,,0
21597,21598,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,90,...,,0.0,,0.0,,0,,0,,0
21598,21599,,,1,40000,R12,P6.2 - Varejo Inativos,z. Verificar,1,91,...,,0.0,,0.0,,0,,0,,0


In [13]:
data_id = 6002
print(json.dumps(listaResultado[data_id-1],indent=4))

{
    "marcacaoMesa": "",
    "etapa": "",
    "flagErroBacen": false,
    "flagErroSerasa": false,
    "flagErroAurora": false,
    "flagErroVisaoCliente": false,
    "flagErroReceitaFederal": false,
    "flagErroCredilink": false,
    "limitePoliticaConcessao": -1,
    "mensagemFiltro": "99. Accepted",
    "mensagemFiltroPolitica": "99. Accepted",
    "mensagemFiltroFinal": "99. Accepted",
    "solicitante": {
        "credilink": {
            "situacaoDocumento": "REGULAR",
            "obito": "NAO"
        },
        "limites": {
            "limiteDisponivel": 10000,
            "valorContratadoCartao": 0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0,
            "vlrMaxRestritivosBVSM2": 0,
            "vlrMaxRestritivosBVSM3": 0
        },
        "ghAppcardsBl

## <b> Teste casos especificos

In [3]:
#Criação das massas com casos forçados
massa_TabelaJaCliente1 = sherlock.Gera_Massa(base_entrada)
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.faixaRiscoInternoAgrupada',["R2"])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.segmentacaoPolitica',["P1 - Funcionarios"])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.faixaRendaBruta',[
                                                                                  'a. < 4k'
                                                                                 ])
massa_TabelaJaCliente1.gera_cenario('solicitante.flagFuncionario',[1])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.cpf6e7Digito',[0,43,44,45,46,88,89,90,91,99])
massa_TabelaJaCliente1.gera_cenario('solicitante.flagCCRot',[1])


massa_TabelaJaCliente1.set_bom_arqv()
massa_TabelaJaCliente1.converte_massa()

listaResultado = rp.execucaoModular(massa_TabelaJaCliente1.get_massa_json(),rp.subGruposAntigo)

# Renomeação das colunas para o cabeçalho do RMA
massa_TabelaJaCliente1.rename_cabecalho()
massa_TabelaJaCliente1.rename_variavel("solicitante.flagCCRot","payloadParam.solicitante.flagCCRot")
massa_TabelaJaCliente1.rename_variavel("solicitante.mediaSaldoTotal","payloadParam.solicitante.mediaSaldoTotal")


massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacao","segmentacaoSubgrupo")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.alavancagem","alavancagem")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteTeto","limiteTeto")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.limitePiso","limitePiso")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteFixo","limiteFixo")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoBAUAux","aprovadoBAUAux")
massa_TabelaJaCliente1.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoSLAux","aprovadoSmallLimitsAux")

massa_TabelaJaCliente1.get_massa_dm()

--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


,Data-ID,Result-State,Result-Detail,payloadParam.solicitante.mediaSaldoTotal,rangeScoreInterno,payloadParamBTS.segmentacaoPolitica,faixaRendaBruta,payloadParam.solicitante.flagFuncionario,cpf6e7Digito,payloadParam.solicitante.flagCCRot,...,result:limiteTeto,expected:limiteTeto,result:limitePiso,expected:limitePiso,result:limiteFixo,expected:limiteFixo,result:aprovadoBAUAux,expected:aprovadoBAUAux,result:aprovadoSmallLimitsAux,expected:aprovadoSmallLimitsAux
0,1,,,40000,R2,P1 - Funcionarios,a. < 4k,1,0,1,...,,3200.0,,300.0,,0,,1,,0
1,2,,,40000,R2,P1 - Funcionarios,a. < 4k,1,43,1,...,,3200.0,,300.0,,0,,1,,0
2,3,,,40000,R2,P1 - Funcionarios,a. < 4k,1,44,1,...,,3200.0,,300.0,,0,,1,,0
3,4,,,40000,R2,P1 - Funcionarios,a. < 4k,1,45,1,...,,3200.0,,300.0,,0,,1,,0
4,5,,,40000,R2,P1 - Funcionarios,a. < 4k,1,46,1,...,,3200.0,,300.0,,0,,1,,0
5,6,,,40000,R2,P1 - Funcionarios,a. < 4k,1,88,1,...,,3200.0,,300.0,,0,,1,,0
6,7,,,40000,R2,P1 - Funcionarios,a. < 4k,1,89,1,...,,3200.0,,300.0,,0,,1,,0
7,8,,,40000,R2,P1 - Funcionarios,a. < 4k,1,90,1,...,,3200.0,,300.0,,0,,1,,0
8,9,,,40000,R2,P1 - Funcionarios,a. < 4k,1,91,1,...,,3200.0,,300.0,,0,,1,,0
9,10,,,40000,R2,P1 - Funcionarios,a. < 4k,1,99,1,...,,3200.0,,300.0,,0,,1,,0


## QUEBRA DA MASSA EM PEDAÇOS

In [14]:


# dataframeGrande

num_dataframes = 2

split_dataframes = np.array_split(dataframeGrande, num_dataframes)

for i, df in enumerate(split_dataframes):
    df.to_csv(f'Massas/Modulares/massaDefinicaoTabelasSubGruposAntigosV2-part{i+1}.csv', index=False)

c:\Users\T51657\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
